# Data Processing / Wrangling / Feature Engineering  

Data collection has been performed using a web scraper designed with a proxy crawler to avoid IP ban, which was later removed due to various test results showing that their firewall does not flag any intensive pings/requests coming from the same IP address on their server in a humanly-not-possible way. The target of the scraping script was `propertyfinder.qa`, which was deemed as the richest in terms of possible observations for rent offers in Qatar. The web scraper script contains some feature preprocessing, but most of it is done here. 

## Collection

In [507]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import random
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import iqr

In [508]:
dataset = pd.read_csv("final.csv")

In [509]:
dataset.head()

,Price,Type,Area(sqm),NoBedrooms,NoBathrooms,Furnishing,Amenities,Location
0,2500.0,Apartment,36.0,studio,1,Furnished,"Kitchen Appliances,Pets Allowed,Security",Al Sakhama
1,8000.0,Apartment,60.0,studio,1,Furnished,"Balcony,Built in Wardrobes,Central A/C,Kitchen...",The Pearl
2,8100.0,Apartment,60.0,studio,1,Furnished,"Balcony,Built in Wardrobes,Central A/C,Kitchen...",The Pearl
3,6000.0,Apartment,40.0,studio,1,Furnished,"Built in Wardrobes,Central A/C,Shared Gym,Shar...",West Bay
4,4200.0,HotelApartments,60.0,studio,1,Furnished,"Covered Parking,Security,Shared Gym,Shared Spa",Musheireb


In [510]:
dataset.shape

(8853, 8)

We scraped around 280 pages at an average of 25 entries per page. That's around 7000 observations, and here it appears we have 8000+, something is fishy. We'll inspect for empty observations, and drop them.

In [511]:
dataset.describe()

,Price,Area(sqm)
count,6.635000e+03,6635.000000
mean,1.359343e+04,184.595328
std,1.455306e+05,151.748720
min,2.800000e+01,1.000000
25%,6.500000e+03,100.000000
50%,9.500000e+03,144.000000
75%,1.292500e+04,230.000000
max,1.050000e+07,4480.000000


In [512]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8853 entries, 0 to 8852
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Price        6635 non-null   float64
 1   Type         6635 non-null   object 
 2   Area(sqm)    6635 non-null   float64
 3   NoBedrooms   6635 non-null   object 
 4   NoBathrooms  6605 non-null   object 
 5   Furnishing   5840 non-null   object 
 6   Amenities    5840 non-null   object 
 7   Location     6635 non-null   object 
dtypes: float64(2), object(6)
memory usage: 553.4+ KB


In [513]:
dataset.dropna(subset = ['Price'], inplace = True)

We fill the null values using a backfill method, although the method is not extremely relevant, since we only have missing data for furnishing and amenities, which are not the most important features.

In [514]:
dataset = dataset.fillna(method='backfill')

In [515]:
 print(np.sum(np.sum(dataset.isnull())))

0


In [516]:
dataset.shape

(6635, 8)

So, the collected data comprises 6635 observations from rent listings. To maximize the accuracy of predictions, several most important features have been collected, as it follows:

- `Price`: rent price - QAR/Monthly

- `Type`: segment of establishment - Apartment, Villa, Penthouse, etc. - important to mention here, there were two types ignored in the collected data - Whole Building and Bulk Rent, which were deemed unfit for the purpose of this price estimator and also had insufficient listings

- `Area(sqm)`: area of establishment in square meters - preprocessed format in the web scraping script

- `Bedrooms`/`Bathrooms`: # of bedrooms/bathrooms

- `Location`: preprocessed location of establishment - the zone/neighborhood

- `Amenities`: preprocessed string separated by commas of amenities corresponding to the establishment - Balcony/Security/View at water, etc

- `Furnishing`: level of furnishing in establishment - Furnished/Partly Furnished/Unfurnished


To start with, we should deal with bedrooms and bathrooms. From the analysis of possible values `Bedrooms` column can take, two possible "edge cases" were identified. First, some establishments have enlisted more than 7 bedrooms - `7+`, and thus a random int of 8 or 9 is used instead to apply a bit more randomness and deal with potential bias - the same goes for the number of bathrooms. Bedrooms, though, have another edge case, in which there is a maid room included, which is treated as an extra room.

In [517]:
# bedrooms handling
data = dataset.replace({'NoBedrooms':'studio'},'1').replace({'NoBedrooms':'7+'}, str(random.randint(8,9))).replace({'NoBedrooms':'7++Maid'}, str(random.randint(9,10))).replace({'NoBedrooms':'studio+Maid'}, '2').replace({'NoBedrooms':'1+Maid'},'2').replace({'NoBedrooms':'2+Maid'},'3').replace({'NoBedrooms':'3+Maid'},'4').replace({'NoBedrooms':'4+Maid'},'5').replace({'NoBedrooms':'5+Maid'},'6').replace({'NoBedrooms':'6+Maid'},'7').replace({'NoBedrooms':'7+Maid'},'8')

data['NoBedrooms']= data['NoBedrooms'].map(lambda x: int(x))
      
            
# bathrooms handling
data = data.replace({'NoBathrooms':'7+'}, str(random.randint(8,9))) 
data['NoBathrooms'] =  data['NoBathrooms'].map(lambda no: int(no), na_action = 'ignore')

In [518]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6635 entries, 0 to 8852
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Price        6635 non-null   float64
 1   Type         6635 non-null   object 
 2   Area(sqm)    6635 non-null   float64
 3   NoBedrooms   6635 non-null   int64  
 4   NoBathrooms  6635 non-null   int64  
 5   Furnishing   6635 non-null   object 
 6   Amenities    6635 non-null   object 
 7   Location     6635 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 466.5+ KB


In [519]:
print(np.sum(np.sum(data.isnull())))

0


For amenities, we first convert the comma separated string in a list. A full set of amenities is also welcome to have at hand.

In [520]:
# amenities
data['Amenities'] = data['Amenities'].map(lambda x: x.split(','))
    
data['Total Amenities'] = data['Amenities'].map(lambda x: len(x))
am = []
for fact in data['Amenities']:
    for x in fact: 
        if x not in am: am.append(x)
        
print(am)
print('\n')
data

['Kitchen Appliances', 'Pets Allowed', 'Security', 'Balcony', 'Built in Wardrobes', 'Central A/C', 'Shared Gym', 'Shared Pool', 'Shared Spa', 'View of Water', 'Covered Parking', "Children's Play Area", 'Concierge', 'Walk-in Closet', 'Maid Service', 'Private Jacuzzi', 'View of Landmark', 'Study', "Children's Pool", 'Lobby in Building', 'Networked', 'Maids Room', 'Private Gym', 'Private Pool', 'Private Garden', 'Barbecue Area']




,Price,Type,Area(sqm),NoBedrooms,NoBathrooms,Furnishing,Amenities,Location,Total Amenities
0,2500.0,Apartment,36.0,1,1,Furnished,"[Kitchen Appliances, Pets Allowed, Security]",Al Sakhama,3
1,8000.0,Apartment,60.0,1,1,Furnished,"[Balcony, Built in Wardrobes, Central A/C, Kit...",The Pearl,10
2,8100.0,Apartment,60.0,1,1,Furnished,"[Balcony, Built in Wardrobes, Central A/C, Kit...",The Pearl,10
3,6000.0,Apartment,40.0,1,1,Furnished,"[Built in Wardrobes, Central A/C, Shared Gym, ...",West Bay,4
4,4200.0,HotelApartments,60.0,1,1,Furnished,"[Covered Parking, Security, Shared Gym, Shared...",Musheireb,4
...,...,...,...,...,...,...,...,...,...
8846,13000.0,Villa,500.0,8,9,Unfurnished,"[Balcony, Children's Play Area, Maids Room]",Al Rayyan,3
8847,20000.0,Villa,600.0,9,9,Unfurnished,"[Balcony, Children's Play Area, Maids Room]",Al Luqta,3
8848,14000.0,Villa,400.0,9,6,Unfurnished,"[Maids Room, Pets Allowed, Private Garden]",Al Sailiya,3
8850,12000.0,Villa,450.0,8,9,Partly furnished,"[Balcony, Central A/C, Kitchen Appliances, Mai...",Umm Salal Mohammad,7


After extensive research (I mean it), it has been noticed that the conversion of `Furnishing` into dummies - OneHotEncoding returns less exciting results, while encoding using a simple logical ranking of

1. Furnished - 3, the most expensive
2. Partly furnished - 2, middle grounds
3. Unfurnished - 1, cheapest

returned the best results.

In [521]:
types_furnish = []
for x in data['Furnishing']:
    if x not in types_furnish: types_furnish.append(x)
print(types_furnish)

['Furnished', 'Unfurnished', 'Partly furnished']


In [522]:
furn_codes = [3,2,1]
dict_furn = dict(zip(types_furnish,furn_codes))
furn_encoding = {'Furnishing': dict_furn}
# print(loc_encoding)
data = data.replace(furn_encoding)

In [523]:
data

,Price,Type,Area(sqm),NoBedrooms,NoBathrooms,Furnishing,Amenities,Location,Total Amenities
0,2500.0,Apartment,36.0,1,1,3,"[Kitchen Appliances, Pets Allowed, Security]",Al Sakhama,3
1,8000.0,Apartment,60.0,1,1,3,"[Balcony, Built in Wardrobes, Central A/C, Kit...",The Pearl,10
2,8100.0,Apartment,60.0,1,1,3,"[Balcony, Built in Wardrobes, Central A/C, Kit...",The Pearl,10
3,6000.0,Apartment,40.0,1,1,3,"[Built in Wardrobes, Central A/C, Shared Gym, ...",West Bay,4
4,4200.0,HotelApartments,60.0,1,1,3,"[Covered Parking, Security, Shared Gym, Shared...",Musheireb,4
...,...,...,...,...,...,...,...,...,...
8846,13000.0,Villa,500.0,8,9,2,"[Balcony, Children's Play Area, Maids Room]",Al Rayyan,3
8847,20000.0,Villa,600.0,9,9,2,"[Balcony, Children's Play Area, Maids Room]",Al Luqta,3
8848,14000.0,Villa,400.0,9,6,2,"[Maids Room, Pets Allowed, Private Garden]",Al Sailiya,3
8850,12000.0,Villa,450.0,8,9,1,"[Balcony, Central A/C, Kitchen Appliances, Mai...",Umm Salal Mohammad,7


In [524]:
#furnished dummies
# df_dummy = pd.get_dummies(data['Furnishing'])
# print('\n')

# df_main_drop = data.drop(columns=['Furnishing'], inplace=False)
# df_main_new = pd.concat([df_main_drop, df_dummy], axis=1, join='inner')
# df_main_new.head(5)

In [525]:
# data = df_main_new

For `Type` on the contrary, the best results have been returned using `one-hot encoding`. Here, there were multiple attempts of dealing with this categorical variable, using for example an encoding after this algorithm:

1. Take the average prices for each type
2. Sort those prices 
3. Rank the types by their average prices and replace the types with those ranks

but with poor results compared to getting dummies.

In [526]:
df_dummy_type = pd.get_dummies(data['Type'])
print('\n')

df_main_drop = data.drop(columns=['Type'], inplace=False)
df_main_new = pd.concat([df_main_drop, df_dummy_type], axis=1, join='inner')
df_main_new.head(5)

,Price,Area(sqm),NoBedrooms,NoBathrooms,Furnishing,Amenities,Location,Total Amenities,Apartment,Compound,Duplex,HotelApartments,Penthouse,Townhouse,Villa
0,2500.0,36.0,1,1,3,"[Kitchen Appliances, Pets Allowed, Security]",Al Sakhama,3,1,0,0,0,0,0,0
1,8000.0,60.0,1,1,3,"[Balcony, Built in Wardrobes, Central A/C, Kit...",The Pearl,10,1,0,0,0,0,0,0
2,8100.0,60.0,1,1,3,"[Balcony, Built in Wardrobes, Central A/C, Kit...",The Pearl,10,1,0,0,0,0,0,0
3,6000.0,40.0,1,1,3,"[Built in Wardrobes, Central A/C, Shared Gym, ...",West Bay,4,1,0,0,0,0,0,0
4,4200.0,60.0,1,1,3,"[Covered Parking, Security, Shared Gym, Shared...",Musheireb,4,0,0,0,1,0,0,0


In [527]:
data = df_main_new

For `Location`, a lot of back-and-forth testing took place. The best results were returned by encoding with a shuffling of the ordering of locations and attributing an integer from 1 to 72 to each of them (there are 72 different locations). A few of the attempted methods:

1. One-hot encoding - disasterous results
2. Using a similar method as before with `Type`, taking the average price for each location and ranking them - encoding using the given ranking - poor results, yet better than getting dummies
3. Encoding using the standard long/lat coordinates for each location, attempted in 2 ways - replacing the `Location` column with 2 new columns, one for longitude, one for latitude, or forming some kind of encryption between long and lat for each location value and replacing `Location` with that - poor results again, yet better than method 2)
4. Encoding `Location` with some sort of random cypher, a hashing function. This method proved also inefficient.

This paragraph has been written to counteract any assumption that encoding randomly with numbers from 1 to 72 will be biased and not give good results. 


In [479]:
# df_dummy_type = pd.get_dummies(data['Location'])
# print('\n')

# df_main_drop = data.drop(columns=['Location'], inplace=False)
# df_main_new = pd.concat([df_main_drop, df_dummy_type], axis=1, join='inner')
# df_main_new.head(5)

In [480]:
# data = df_main_new

In [481]:
# from functools import reduce
# def get_avg(l):
#     meow = list(data['Price'].loc[data['Location'] == l])
# #     print(meow)
#     avg = reduce((lambda x, y: x + y), meow) / len(meow)
#     return avg
# locs = []
# for x in data['Location']:
#     if x not in locs: locs.append(x)
# print(locs)
# # print(get_avg(locs[0]))
# loc_codes = [get_avg(x)/100 for x in locs]
# print(loc_codes)

In [528]:
#location encoding
locs = []
for x in data['Location']:
    if x not in locs: locs.append(x)

loc_codes = [x for x in range(1,73)]
dict_locs = dict(zip(locs,loc_codes))

######################################testing
# dict_locs = {'Al Beshairiya Street': 0, 'Al Hitmi': 1, 'Industrial Area': 2, 'Fereej Abdul Aziz': 3, 'Mughalina': 4, 'New Doha': 5, 'Doha Al Jadeed': 6, 'Umm Ghuwailina': 7, 'Mesaimeer': 8, 'Fereej Kulaib': 9, 'Al Ghanim': 10, 'Najma': 11, 'Al Asmakh': 12, 'D-Ring': 13, 'Al Mansoura': 14, 'B-Ring Road': 15, 'Al Jebailat': 16, 'Al Muntazah': 17, 'Umm Salal Ali': 18, 'C-Ring': 19, 'Airport Area': 20, 'Al Sakhama': 21, 'Al Aziziyah': 22, 'Al Sadd': 23, 'Rawdat Al Khail': 24, 'Musheireb': 25, 'Rawdat Al Matar': 26, 'Old Airport Road': 27, 'Al Nasr': 28, 'Fereej Bin Omran': 29, 'Al Mirqab': 30, 'Umm Salal Mohammad': 31, 'Salata': 32, 'Barwa City': 33, 'Fereej Bin Mahmoud': 34, 'Salwa Road': 35, 'Corniche Road': 36, 'Al Thumama': 37, 'Al Rawda': 38, 'Jeliah': 39, 'Izghawa': 40, 'Muaither Area': 41, 'Al Gharrafa': 42, 'Abu Hamour': 43, 'Al Maamoura': 44, 'Al Rayyan': 45, 'Madinat Khalifa': 46, 'Al Hilal': 47, 'Al Luqta': 48, 'Al Soudan': 49, 'Al Duhail': 50, 'AlMuraikh': 51, 'Al Nuaija': 52, 'Al Jasra': 53, 'Al Markhiya': 54, 'Msheireb Downtown Doha': 55, 'Al Waab': 56, 'Fereej Al Ali': 57, 'West Bay': 58, 'Diplomats Area': 59, 'Umm Al Seneem': 60, 'Al Sailiya': 61, 'Al Dafna': 62, 'Al Messila': 63, 'Al Asiri': 64, 'Business District': 65, 'Onaiza': 66, 'The Pearl': 67, 'Ain Khaled': 68, 'Onaiza 65': 69, 'West Bay Lagoon': 70, 'Hazm Al Markhiya': 71}

loc_encoding = {'Location': dict_locs}
print(loc_encoding)
data = data.replace(loc_encoding)
data

{'Location': {'Al Sakhama': 1, 'The Pearl': 2, 'West Bay': 3, 'Musheireb': 4, 'Old Airport Road': 5, 'Al Duhail': 6, 'Mesaimeer': 7, 'Umm Salal Mohammad': 8, 'Al Aziziyah': 9, 'Al Mansoura': 10, 'Al Sadd': 11, 'Al Beshairiya Street': 12, 'Muaither Area': 13, 'Al Ghanim': 14, 'Umm Salal Ali': 15, 'Umm Ghuwailina': 16, 'Al Gharrafa': 17, 'Salata': 18, 'Al Maamoura': 19, 'Najma': 20, 'Al Markhiya': 21, 'Jeliah': 22, 'West Bay Lagoon': 23, 'Ain Khaled': 24, 'Corniche Road': 25, 'Madinat Khalifa': 26, 'Doha Al Jadeed': 27, 'Al Dafna': 28, 'Fereej Bin Mahmoud': 29, 'Al Waab': 30, 'Al Muntazah': 31, 'Al Thumama': 32, 'Abu Hamour': 33, 'Al Nasr': 34, 'Salwa Road': 35, 'Izghawa': 36, 'Al Asmakh': 37, 'Fereej Abdul Aziz': 38, 'Al Rawda': 39, 'Fereej Bin Omran': 40, 'Al Jasra': 41, 'Al Jebailat': 42, 'Diplomats Area': 43, 'Msheireb Downtown Doha': 44, 'Rawdat Al Khail': 45, 'Al Hilal': 46, 'Onaiza': 47, 'Al Messila': 48, 'Al Nuaija': 49, 'Mughalina': 50, 'B-Ring Road': 51, 'Al Rayyan': 52, 'Al Mi

,Price,Area(sqm),NoBedrooms,NoBathrooms,Furnishing,Amenities,Location,Total Amenities,Apartment,Compound,Duplex,HotelApartments,Penthouse,Townhouse,Villa
0,2500.0,36.0,1,1,3,"[Kitchen Appliances, Pets Allowed, Security]",1,3,1,0,0,0,0,0,0
1,8000.0,60.0,1,1,3,"[Balcony, Built in Wardrobes, Central A/C, Kit...",2,10,1,0,0,0,0,0,0
2,8100.0,60.0,1,1,3,"[Balcony, Built in Wardrobes, Central A/C, Kit...",2,10,1,0,0,0,0,0,0
3,6000.0,40.0,1,1,3,"[Built in Wardrobes, Central A/C, Shared Gym, ...",3,4,1,0,0,0,0,0,0
4,4200.0,60.0,1,1,3,"[Covered Parking, Security, Shared Gym, Shared...",4,4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8846,13000.0,500.0,8,9,2,"[Balcony, Children's Play Area, Maids Room]",52,3,0,0,0,0,0,0,1
8847,20000.0,600.0,9,9,2,"[Balcony, Children's Play Area, Maids Room]",66,3,0,0,0,0,0,0,1
8848,14000.0,400.0,9,6,2,"[Maids Room, Pets Allowed, Private Garden]",72,3,0,0,0,0,0,0,1
8850,12000.0,450.0,8,9,1,"[Balcony, Central A/C, Kitchen Appliances, Mai...",8,7,0,0,0,0,0,0,1


In [483]:
print(len(am))

26


For `Amenities`, we need to separate those amenities from the lists and form dummies for each of them. With small assistance from StackOverflow at a basic "How to" google search, the `MultiLabelBinarizer` method will be best fit for this case.

In [529]:
# a little help from StackOverflow

mlb = MultiLabelBinarizer()
dummies_amenities = pd.DataFrame(mlb.fit_transform(data['Amenities']),columns=mlb.classes_, index=data.index)
data = data.drop(columns = ['Amenities'])
data = pd.concat([data, dummies_amenities], axis =1)

In [530]:
data.info()
# print(types)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6635 entries, 0 to 8852
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Price                 6635 non-null   float64
 1   Area(sqm)             6635 non-null   float64
 2   NoBedrooms            6635 non-null   int64  
 3   NoBathrooms           6635 non-null   int64  
 4   Furnishing            6635 non-null   int64  
 5   Location              6635 non-null   int64  
 6   Total Amenities       6635 non-null   int64  
 7   Apartment             6635 non-null   uint8  
 8   Compound              6635 non-null   uint8  
 9   Duplex                6635 non-null   uint8  
 10  HotelApartments       6635 non-null   uint8  
 11  Penthouse             6635 non-null   uint8  
 12  Townhouse             6635 non-null   uint8  
 13  Villa                 6635 non-null   uint8  
 14  Balcony               6635 non-null   int32  
 15  Barbecue Area        

In [531]:
print(loc_encoding)

{'Location': {'Al Sakhama': 1, 'The Pearl': 2, 'West Bay': 3, 'Musheireb': 4, 'Old Airport Road': 5, 'Al Duhail': 6, 'Mesaimeer': 7, 'Umm Salal Mohammad': 8, 'Al Aziziyah': 9, 'Al Mansoura': 10, 'Al Sadd': 11, 'Al Beshairiya Street': 12, 'Muaither Area': 13, 'Al Ghanim': 14, 'Umm Salal Ali': 15, 'Umm Ghuwailina': 16, 'Al Gharrafa': 17, 'Salata': 18, 'Al Maamoura': 19, 'Najma': 20, 'Al Markhiya': 21, 'Jeliah': 22, 'West Bay Lagoon': 23, 'Ain Khaled': 24, 'Corniche Road': 25, 'Madinat Khalifa': 26, 'Doha Al Jadeed': 27, 'Al Dafna': 28, 'Fereej Bin Mahmoud': 29, 'Al Waab': 30, 'Al Muntazah': 31, 'Al Thumama': 32, 'Abu Hamour': 33, 'Al Nasr': 34, 'Salwa Road': 35, 'Izghawa': 36, 'Al Asmakh': 37, 'Fereej Abdul Aziz': 38, 'Al Rawda': 39, 'Fereej Bin Omran': 40, 'Al Jasra': 41, 'Al Jebailat': 42, 'Diplomats Area': 43, 'Msheireb Downtown Doha': 44, 'Rawdat Al Khail': 45, 'Al Hilal': 46, 'Onaiza': 47, 'Al Messila': 48, 'Al Nuaija': 49, 'Mughalina': 50, 'B-Ring Road': 51, 'Al Rayyan': 52, 'Al Mi

Change all the data types in the dataset to `int` for uniformity.

In [532]:
data['Price'] = data['Price'].map(lambda x: int(x))
data['Area(sqm)'] = data['Area(sqm)'].map(lambda x: int(x))

In [533]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6635 entries, 0 to 8852
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Price                 6635 non-null   int64
 1   Area(sqm)             6635 non-null   int64
 2   NoBedrooms            6635 non-null   int64
 3   NoBathrooms           6635 non-null   int64
 4   Furnishing            6635 non-null   int64
 5   Location              6635 non-null   int64
 6   Total Amenities       6635 non-null   int64
 7   Apartment             6635 non-null   uint8
 8   Compound              6635 non-null   uint8
 9   Duplex                6635 non-null   uint8
 10  HotelApartments       6635 non-null   uint8
 11  Penthouse             6635 non-null   uint8
 12  Townhouse             6635 non-null   uint8
 13  Villa                 6635 non-null   uint8
 14  Balcony               6635 non-null   int32
 15  Barbecue Area         6635 non-null   int32
 16  Built 

Better later than never, it's time to check for outliers. A simple rule and widely encountered option for removing outliers that was used here was dropping observations that are more than 3 standard deviations away from the mean. Other oulier removal procedures were attempted and tested yet the one proposed has demonstrated to be the most efficient.

In [538]:
# removing outliers

possible_outliers = data[["Price","Area(sqm)"]]
out_test = data.drop(columns = ["Price","Area(sqm)"])

# 3 standard deviations cutoff -> above or below is outlier

meann = np.mean(possible_outliers)
stdd = np.std(possible_outliers)
outs = possible_outliers[(possible_outliers > meann + 3 * stdd) | (possible_outliers < meann - 3 * stdd)]
clean = possible_outliers[ (possible_outliers > meann- 3 * stdd) & (possible_outliers < meann + 3 * stdd)]

In [539]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6635 entries, 0 to 8852
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Price      6629 non-null   float64
 1   Area(sqm)  6543 non-null   float64
dtypes: float64(2)
memory usage: 155.5 KB


In [540]:
data_clean = pd.concat([clean, out_test], axis=1, sort=False)
data_clean.dropna(subset = ['Price'], inplace = True)
data_clean.dropna(subset = ['Area(sqm)'], inplace = True)

data_clean

,Price,Area(sqm),NoBedrooms,NoBathrooms,Furnishing,Location,Total Amenities,Apartment,Compound,Duplex,...,Private Jacuzzi,Private Pool,Security,Shared Gym,Shared Pool,Shared Spa,Study,View of Landmark,View of Water,Walk-in Closet
0,2500.0,36.0,1,1,3,1,3,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,8000.0,60.0,1,1,3,2,10,1,0,0,...,0,0,1,1,1,1,0,0,1,0
2,8100.0,60.0,1,1,3,2,10,1,0,0,...,0,0,1,1,1,1,0,0,1,0
3,6000.0,40.0,1,1,3,3,4,1,0,0,...,0,0,0,1,1,0,0,0,0,0
4,4200.0,60.0,1,1,3,4,4,0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8834,14000.0,313.0,8,9,2,32,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8846,13000.0,500.0,8,9,2,52,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8847,20000.0,600.0,9,9,2,66,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8848,14000.0,400.0,9,6,2,72,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [541]:
data = data_clean

In [542]:
data['Price'] = data['Price'].map(lambda x: int(x))
data['Area(sqm)'] = data['Area(sqm)'].map(lambda x: int(x))

In [543]:
data

,Price,Area(sqm),NoBedrooms,NoBathrooms,Furnishing,Location,Total Amenities,Apartment,Compound,Duplex,...,Private Jacuzzi,Private Pool,Security,Shared Gym,Shared Pool,Shared Spa,Study,View of Landmark,View of Water,Walk-in Closet
0,2500,36,1,1,3,1,3,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,8000,60,1,1,3,2,10,1,0,0,...,0,0,1,1,1,1,0,0,1,0
2,8100,60,1,1,3,2,10,1,0,0,...,0,0,1,1,1,1,0,0,1,0
3,6000,40,1,1,3,3,4,1,0,0,...,0,0,0,1,1,0,0,0,0,0
4,4200,60,1,1,3,4,4,0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8834,14000,313,8,9,2,32,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8846,13000,500,8,9,2,52,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8847,20000,600,9,9,2,66,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8848,14000,400,9,6,2,72,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [497]:
data.to_csv('data_ready.csv')

In [498]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6538 entries, 0 to 8850
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Price                 6538 non-null   int64
 1   Area(sqm)             6538 non-null   int64
 2   NoBedrooms            6538 non-null   int64
 3   NoBathrooms           6538 non-null   int64
 4   Furnishing            6538 non-null   int64
 5   Location              6538 non-null   int64
 6   Total Amenities       6538 non-null   int64
 7   Apartment             6538 non-null   uint8
 8   Compound              6538 non-null   uint8
 9   Duplex                6538 non-null   uint8
 10  HotelApartments       6538 non-null   uint8
 11  Penthouse             6538 non-null   uint8
 12  Townhouse             6538 non-null   uint8
 13  Villa                 6538 non-null   uint8
 14  Balcony               6538 non-null   int32
 15  Barbecue Area         6538 non-null   int32
 16  Built 

In [552]:
# to be included in next notebook
print(locs)
print("\n")
print(types_furnish)
print("\n")
print(am)


['Al Sakhama', 'The Pearl', 'West Bay', 'Musheireb', 'Old Airport Road', 'Al Duhail', 'Mesaimeer', 'Umm Salal Mohammad', 'Al Aziziyah', 'Al Mansoura', 'Al Sadd', 'Al Beshairiya Street', 'Muaither Area', 'Al Ghanim', 'Umm Salal Ali', 'Umm Ghuwailina', 'Al Gharrafa', 'Salata', 'Al Maamoura', 'Najma', 'Al Markhiya', 'Jeliah', 'West Bay Lagoon', 'Ain Khaled', 'Corniche Road', 'Madinat Khalifa', 'Doha Al Jadeed', 'Al Dafna', 'Fereej Bin Mahmoud', 'Al Waab', 'Al Muntazah', 'Al Thumama', 'Abu Hamour', 'Al Nasr', 'Salwa Road', 'Izghawa', 'Al Asmakh', 'Fereej Abdul Aziz', 'Al Rawda', 'Fereej Bin Omran', 'Al Jasra', 'Al Jebailat', 'Diplomats Area', 'Msheireb Downtown Doha', 'Rawdat Al Khail', 'Al Hilal', 'Onaiza', 'Al Messila', 'Al Nuaija', 'Mughalina', 'B-Ring Road', 'Al Rayyan', 'Al Mirqab', 'Industrial Area', 'Al Hitmi', 'New Doha', 'Airport Area', 'AlMuraikh', 'D-Ring', 'Barwa City', 'C-Ring', 'Fereej Kulaib', 'Rawdat Al Matar', 'Al Soudan', 'Business District', 'Al Luqta', 'Umm Al Seneem', 